In [89]:
! pip install bs4
! pip install selenium
! pip install webdriver_manager

  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
import numpy as np
import pandas as pd
import time
from datetime import date
import bs4
from bs4 import BeautifulSoup
import csv
import requests as rqt
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [ ]:
data = pd.read_csv(r"/content/Amazon Scraping - Sheet1.csv")
data

,Unnamed: 0,id,Asin,country
0,0,1,1015,de
1,1,2424796,1015,fr
2,2,2,000004458X,de
3,3,2424797,000004458X,fr
4,4,3,1002198,de
...,...,...,...,...
995,995,2425286,4484924,fr
996,996,449,4485742,de
997,997,2425287,4485742,fr
998,998,450,4486072,de


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   id          1000 non-null   int64 
 2   Asin        1000 non-null   object
 3   country     1000 non-null   object
dtypes: int64(2), object(2)
memory usage: 31.4+ KB


In [ ]:
data['Asin'] = data['Asin'].replace('X','').astype(int)
data['country'] = data['country'].astype(str)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   id          1000 non-null   int64 
 2   Asin        1000 non-null   int64 
 3   country     1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [ ]:
data.describe()

,Unnamed: 0,id,Asin
count,1000.000000,1.000000e+03,1.000000e+03
mean,499.500000,1.502549e+06,2.544850e+06
std,288.819436,1.531627e+06,1.658578e+06
min,0.000000,1.000000e+00,1.015000e+03
25%,249.750000,2.507500e+02,1.084682e+06
50%,499.500000,2.424846e+06,1.199774e+06
75%,749.250000,2.425095e+06,4.173325e+06
max,999.000000,7.536534e+06,4.486072e+06


In [ ]:
data.shape

(1000, 4)

In [ ]:
print(data)

     Unnamed: 0       id     Asin country
0             0        1     1015      de
1             1  2424796     1015      fr
2             2        2     4458      de
3             3  2424797     4458      fr
4             4        3  1002198      de
..          ...      ...      ...     ...
995         995  2425286  4484924      fr
996         996      449  4485742      de
997         997  2425287  4485742      fr
998         998      450  4486072      de
999         999  2425288  4486072      fr

[1000 rows x 4 columns]


In [ ]:
Asin = data.iloc[0:999,2:3].values
Asin


array([[   1015],
       [   1015],
       [   4458],
       [   4458],
       [1002198],
       [1002198],
       [1002791],
       [1002791],
       [1002864],
       [1002864],
       [1003704],
       [1003704],
       [1003763],
       [1003763],
       [1004271],
       [1004271],
       [ 101742],
       [ 101742],
       [1017519],
       [1017519],
       [ 102163],
       [ 102163],
       [1022369],
       [1022369],
       [1022857],
       [1022857],
       [1032666],
       [1032666],
       [1034677],
       [1034677],
       [1034936],
       [1034936],
       [1034944],
       [1034944],
       [1035002],
       [1035002],
       [1035029],
       [1035029],
       [1035053],
       [1035053],
       [1035053],
       [1035339],
       [1035339],
       [1036866],
       [1036866],
       [1036866],
       [1037137],
       [1037137],
       [1037188],
       [1037188],
       [1037994],
       [1037994],
       [ 103863],
       [ 103863],
       [1039466],
       [10

In [76]:
# Opening the search enggine
# Search for the varies products
# Extract the html content of all the products
# Extract the product description, price, ratings, reviews count and URL
# Record the product information in a product record list
# Repeat for all the pages
# Close the browser
# Write all the product's information in the product record list in the spreadsheet

class AmazonProductScraper():
  def __init__(self):#,driver,category_name,formatted_category_name):
    self.driver = None#driver
    self.category_name = None#category_name
    self.formatted_category_name = None#formatted_category_name

  def open_browser(self):
    opt = Options()
    opt.add_argument("--disable-infobars")
    opt.add_argument("--disable-extensions")
    opt.add_argument('--log-level=OFF')
    opt.add_experimental_option('excludeSwitches', ['enable-logging'])

    url = ("https://www.amazon.in/")
    self.driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=opt(url))
    # Website URL
    self.driver.get(url)

    # Wait till the page has been loaded
    time.sleep(3)

In [77]:
def get_category_url(self):
    self.category_name = input("\n>> Enter the product/category to be searched: ")

    self.formatted_category_name = self.category_name.replace(" ", "+")

    # This is the product url format for all products
    category_url = "https://www.amazon.in/s?k={}&ref=nb_sb_noss"

    category_url = category_url.format(self.formatted_category_name)

    print(">> Category URL: ", category_url)

    # Go to the product webpage
    self.driver.get(category_url)
    # To be used later while navigating to different pages
    return category_url

In [78]:
def extract_webpage_information(self):
  # Parsing through the webpage
  soup = BeautifulSoup(self.driver.page_source, 'html.parser')
  # List of all the html information related to the product
  page_results = soup.find_all('div', {'data-component-type': 's-search-result'})
  page_results

In [79]:
@staticmethod
def extract_product_information(page_results):
  temp_record = []
  for i in range(len(page_results)):
    item = page_results[i]

      # Find the a tag of the item
    a_tag_item = item.h2.a

      # Name of the item
    description = a_tag_item.text.strip()

      # Get the url of the item
    category_url = "https://www.amazon.in/" + a_tag_item.get('href')

      # Get the price of the product
    try:
      product_price_location = item.find('span', 'a-price')
      product_price = product_price_location.find('span', 'a-offscreen').text
    except AttributeError:
      product_price = "N/A"

        # Get product reviews
    try:
      product_review = item.i.text.strip()
    except AttributeError:
      product_review = "N/A"

      # Get number of reviews
    try:
      review_number = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
      review_number = "N/A"

      # Store the product information in a tuple
      product_information = (description,  product_price[1:], product_review, review_number, category_url)

      # Store the information in a temporary record
      temp_record.append(product_information)
    
      return temp_record

In [80]:
def navigate_to_other_pages(self, category_url):
    # Contains the list of all the product's information
  records = []

  print("\n>> Page 1 - webpage information extracted")

  try:
    max_number_of_pages = ("//span[@class='s-pagination-item s-pagination-disabled']")
    number_of_pages = self.driver.find_element_by_xpath(max_number_of_pages)
    print("Maximum Pages: ", number_of_pages.text)
  except NoSuchElementException:
    max_number_of_pages = "//li[@class='a-normal'][last()]"
    number_of_pages = self.driver.find_element_by_xpath(max_number_of_pages)
    for i in range(2, int(number_of_pages.text)+1):
        # Goes to next page
      next_page_url = category_url+ "&page=" + str(i)
      self.driver.get(next_page_url)

        # Webpage information is stored in page_results
    page_results = self.extract_webpage_information()
    temp_record = self.extract_product_information(page_results)

    extraction_information = ">> Page {} - webpage information extracted"
    print(extraction_information.format(i))

    for j in temp_record:
      records.append(j)

      self.driver.close()

    print("\n>> Creating an excel sheet and entering the details...")

    return records

In [81]:
def product_information_spreadsheet(self, records):
  today = date.today().strftime("%d-%m-%Y")
  for _ in records:
    file_name = "{}_{}.csv".format(self.category_name, today)
    f = open(file_name, "w", newline='', encoding='utf-8')
    writer = csv.writer(f)
    writer.writerow(['Description', 'Price', 'Rating', 'Review Count', 'Product URL'])
    writer.writerows(records)
    f.close()
    message = (">> Information about the product '{}' is stored in {}\n").format(self.category_name, file_name)
    print(message)

    os.startfile(file_name)

In [88]:
my_amazon_bot = AmazonProductScraper()

my_amazon_bot.open_browser

category_details = my_amazon_bot.get_category_url()

my_amazon_bot.extract_product_information(my_amazon_bot.extract_webpage_information)

navigation = my_amazon_bot.navigate_to_other_pages(category_details)

my_amazon_bot.product_information_spreadsheet(navigation)